Detailed Dashboard
User needs to provide their key.

Differences from Basic:
   Threats can have multiple sources
   Threats can have multiple impacts

In [ ]:
userkey = "140"
dbhost  = 'localhost'

In [ ]:
import pandas as pd
import numpy as np
import scipy as sp
import matplotlib.pyplot as plt
%matplotlib inline
import ipywidgets as widgets
import psycopg2
import qual

conn = psycopg2.connect("postgres://csc452:csc452@" + dbhost + "/csc452")  
cursor = conn.cursor()  
cursor.execute('SET mydomain.userkey = ' + userkey)



triages = pd.read_sql_query ('SELECT name, description, vulnerability, pervasive, impact, impact_name, impact_description, source_name, source_description, capacity, init FROM triagev', conn)
tr = triages.loc[triages['source_name']!='source name']

# still want to calculate the qualitative risk assessments

for i in tr.index:
    v = qual.qdict[qual.norm(tr.at[i,'vulnerability'])]
    l = qual.qdict[qual.norm(tr.at[i,'init'])]
    im = qual.qdict[qual.norm(tr.at[i,'impact'])]
    o = qual.overall[l][v]
    tr.at[i,'overall_likelihood'] = o
    tr.at[i,'risk_level'] = qual.risk_level[o][im]
    tr.at[i, 'risk_num'] = qual.qval_dict[qual.risk_level[o][im]]
    tr.at[i, 'annual_events'] = tr.at[i,'init'] * qual.rate_dict[l]
    tr.at[i, 'event_loss'] = tr.at[i,'impact'] * qual.loss_dict[im]
    tr.at[i, 'annual_loss'] = tr.at[i,'annual_events'] * tr.at[i,'event_loss']

rows = tr.set_index ('name')
rows.sort_values (by=['annual_loss'])

print ("Threat Overview")
rows.filter (items=['name', 'annual_loss', 'risk_level', 'overall_likelihood', 'annual_events', 'event_loss', 'description'])


In [ ]:
print ("Threat Sources")
rows.filter (items=['name', 'source_name', 'source_description', 'init'])

In [ ]:
print ("Threat Impacts")
rows.filter (items=['name', 'impact', 'impact_name', 'impact_description', 'pervasive', 'vulnerability'])